# Chicago Car Crash Dataset

* Student name: Bonny Nichol
* Student pace: Self Paced
* Scheduled project review date/time: November 17, 2020
* Instructor name: Jeff Herman
* Blog post URL:


## Import Libraries

In [1]:
import warnings
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-colorblind')

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree

import xgboost as xgb

/Users/rosegold/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load Data

In [2]:
# load dataset
df = pd.read_csv('crashes.csv')
df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,073682ef84ff827659552d4254ad1b98bfec24935cc9cc...,JB460108,NaN,10/02/2018 06:30:00 PM,10,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,18,3,10,NaN,NaN,NaN
1,1560fb8a1e32b528fef8bfd677d2b3fc5ab37278b157fa...,JC325941,NaN,06/27/2019 04:00:00 PM,45,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,...,0.0,0.0,2.0,0.0,16,5,6,NaN,NaN,NaN
2,c7e667807d86ff2e83da48267e1135680cad68c86eb00f...,JD150617,NaN,02/13/2020 04:30:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,...,0.0,1.0,2.0,0.0,16,5,2,NaN,NaN,NaN
3,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
4,00e47f189660cd8ba1e85fc63061bf1d8465184393f134...,JC194776,NaN,03/21/2019 10:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,...,0.0,0.0,2.0,0.0,22,5,3,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)


## Data Exploration

In [3]:
# Shape of dataset
df.shape

(453873, 49)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453873 entries, 0 to 453872
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                453873 non-null  object 
 1   RD_NO                          450376 non-null  object 
 2   CRASH_DATE_EST_I               33749 non-null   object 
 3   CRASH_DATE                     453873 non-null  object 
 4   POSTED_SPEED_LIMIT             453873 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         453873 non-null  object 
 6   DEVICE_CONDITION               453873 non-null  object 
 7   WEATHER_CONDITION              453873 non-null  object 
 8   LIGHTING_CONDITION             453873 non-null  object 
 9   FIRST_CRASH_TYPE               453873 non-null  object 
 10  TRAFFICWAY_TYPE                453873 non-null  object 
 11  LANE_CNT                       198959 non-null  float64
 12  ALIGNMENT                     

In [5]:
#A heatmap of the missing values visually shows the data that is missing
#sns.heatmap(df.isnull())
#plt.show()

In [6]:
df.columns

Index(['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE',
       'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LA

In [7]:
crashdata.fillna('UNKNOWN', inplace=True)
#crashdata.isnull().any()

NameError: name 'crashdata' is not defined

In [ ]:
crashdata = df.drop(['CRASH_DATE','RD_NO','CRASH_RECORD_ID','CRASH_DATE_EST_I', 'LANE_CNT', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE',
        'WORKERS_PRESENT_I', 'INJURIES_UNKNOWN'], axis=1)
crashdata.info()

In [ ]:
crashdata.isnull().any()

In [ ]:
crashdata.hist(bins=10, figsize=(20,20))

In [ ]:
crashdata['INJURIES_FATAL'].value_counts()

In [ ]:
crashdata = crashdata[crashdata['INJURIES_FATAL'].notna()]
crashdata = crashdata.reset_index()
crashdata



In [ ]:
neigh_map = gpd.read_file('geo_export_aeab7300-7d09-4aec-ad13-b05f99ccc606.shp')
crs = {'init': 'epsg:4326'}

In [ ]:
geometry = [Point(xy) for xy in zip(crashdata['LONGITUDE'], crashdata['LATITUDE'])]
geometry[5:8]
geo_df = gpd.GeoDataFrame(crashdata, crs = crs, geometry = geometry)
geo_data = geo_df.dropna()
geo_data = geo_data[geo_data.LATITUDE > 0]

In [ ]:
#Plotting fatal injuries
fig, ax = plt.subplots(figsize = (10,10))
neigh_map.plot(ax = ax, alpha = 0.4, color='grey')
geo_data[geo_data['INJURIES_FATAL'] == 1.0].plot(ax = ax, markersize = 10, color='blue', marker='o', label = 'Fatal - 1 Person')
geo_data[geo_data['INJURIES_FATAL'] == 2.0].plot(ax = ax, markersize = 10, color='red', marker='o', label = 'Fatal - 2 People')
geo_data[geo_data['INJURIES_FATAL'] == 3.0].plot(ax = ax, markersize = 10, color='orange', marker='o', label = 'Fatal - 3 People')
plt.legend(prop={'size': 10})

In [ ]:
#Plotting non-fatal but serious injuries
fig, ax = plt.subplots(figsize = (10,10))
neigh_map.plot(ax = ax, alpha = 0.4, color='grey')
geo_data[geo_data['INJURIES_INCAPACITATING'] == 1.0].plot(ax = ax, markersize = 10, color='blue', marker='o', label = 'Non-Fatal Serious Injuries')
plt.legend(prop={'size': 10})

In [ ]:
#Plotting non-fatal and non-serious injuries
fig, ax = plt.subplots(figsize = (10,10))
neigh_map.plot(ax = ax, alpha = 0.4, color='grey')
geo_data[geo_data['INJURIES_NON_INCAPACITATING'] == 1.0].plot(ax = ax, markersize = 10, color='blue', marker='o', label = 'Non-Fatal Non-Serious Injuries')
plt.legend(prop={'size': 10})

In [ ]:
#sns.pairplot(geo_data)

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(12,10))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(crashdata.corr(),
            linewidths=0.1,
            vmax=1.0, 
            square=True, 
            cmap=colormap, 
            linecolor='white', 
            annot=True)
plt.show()

In [ ]:
one_hot_data = pd.get_dummies(crashdata[['WEATHER_CONDITION','LIGHTING_CONDITION']],drop_first=True)
#one_hot_data = pd.get_dummies(crashdata[['INJURIES_FATAL']],drop_first=True)
one_hot_data.info()

In [ ]:
# Create features and labels

X = one_hot_data
y = crashdata['INJURIES_FATAL']
#X = crashdata.drop('INJURIES_FATAL', axis=1)  
#y = crashdata['INJURIES_FATAL'] 


# Perform an 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=10)

In [ ]:
# Train a DT classifier
classifier = DecisionTreeClassifier(random_state=10)  
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions for test data
y_pred = classifier.predict(X_test) 

In [ ]:
# Calculate accuracy 
acc = accuracy_score(y_test,y_pred) * 100
print('Accuracy is :{0}'.format(acc))

# Check the AUC for predictions
#false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
#roc_auc = auc(false_positive_rate, true_positive_rate)
#print('\nAUC is :{0}'.format(round(roc_auc, 2)))

# Create and print a confusion matrix 
print('\nConfusion Matrix')
print('----------------')
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# Alternative confusion matrix
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(classifier, X, y, values_format='.3g')
plt.show()

## Decision Tree Using Entropy

In [ ]:
# Instantiate and fit a DecisionTreeClassifier
classifier_2 = DecisionTreeClassifier(random_state=10, criterion='entropy')  
classifier_2.fit(X_train, y_train)

In [ ]:
# Plot and show decision tree
plt.figure(figsize=(20,20), dpi=500)
tree.plot_tree(classifier_2, 
               feature_names=X.columns,
               class_names=np.unique(y).astype('str'),
               filled=True, rounded=True)
plt.show()